# Develop ML model with MLflow and deploy to Kubernetes

This a notbook which summarise the steps done in the tutorial [Develop ML model with MLflow and deploy to Kubernetes](https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html#develop-ml-model-with-mlflow-and-deploy-to-kubernetes)

## Requirements

* PyENV already installed. Please check the [link section](#links)
* Kind [Cluster](./README.md#cluster)
* A python3.10 virtual environment
* mflow on the python virtual environment
* ipykernel

## Training the Model

In [ ]:
import mlflow

import numpy as np
from sklearn import datasets, metrics
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
from mlflow.client import MlflowClient

In [ ]:

def eval_metrics(pred, actual):
    rmse = np.sqrt(metrics.mean_squared_error(actual, pred))
    mae = metrics.mean_absolute_error(actual, pred)
    r2 = metrics.r2_score(actual, pred)
    return rmse, mae, r2

# Load wine quality dataset
X, y = datasets.load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


### Defining a mflow experiment

In [ ]:



# Set th experiment name
mlflow.set_experiment("wine-quality")

# Enable auto-logging to MLflow
mlflow.sklearn.autolog()


# Start a run and train a model
with mlflow.start_run(run_name="default-params"):
    lr = ElasticNet()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)
    metrics = eval_metrics(y_pred, y_test)


### Checking out the experiment

Before running the mflow server ui on port 5000, we checkout if there is a process using port 5000

In case you cannot access to mflow ui on URL [http://localhost:5000](http://localhost:5000). Please check out if there is a process using port 5000 by executing ```lsof -i :5000```

In [ ]:
import subprocess, socket


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    sock.settimeout(5)
    result = sock.connect_ex(('localhost',5000))

if result == 0:
    print('Server already running')
else:
    command = "mlflow ui --port 5000"
    
    process = subprocess.Popen(command, shell=True)
    
    print(f"Started background process with PID: {process.pid}")


### Logging the model

In [ ]:
REGISTERED_MODEL_NAME = "wine-quality"
ARTIFACT_PATH = "wine-quality_model"


# Start an MLflow run
with mlflow.start_run(run_name="logging-model") as run:

    lr = ElasticNet()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("wine-quality", "Basic ElasticNet model for wine-quality")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path=ARTIFACT_PATH,
        signature=signature,
        input_example=X_train,
        registered_model_name=REGISTERED_MODEL_NAME,
    )
    # set extra tags on the model
    client = MlflowClient(mlflow.get_tracking_uri())
    model_info = client.get_latest_versions(REGISTERED_MODEL_NAME)[0]
    client.set_model_version_tag(
        name=REGISTERED_MODEL_NAME,
        version=model_info.version,
        key='model',
        value='ElasticNet'
    )

    print(f'Model Info: {model_info}')


### Testing Model Serving Locally

For this step we need to gather from the artifact section of the latest experiment run.

![image](./image/Screenshot%20from%202024-05-20%2019-35-09.png)

Then we can run the mlserver manually by executing the command below where the last runid is ```ef001c9ef0b3485999157d2ec3d5a2a8```:
```bash
mlflow models serve -m runs:/ef001c9ef0b3485999157d2ec3d5a2a8/wine-quality_model -p 1234 --enable-mlserver
```

Finally, querying the endpoing by using the curl command below:

```bash
curl -X POST -H "Content-Type:application/json" --data '{"inputs": [[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]]}' http://127.0.0.1:1234/invocations
```

We will get the outcome similar to the one below:
```bash
{"predictions": [0.4230039055961139]}
```

Or running the next python cell run launch a mlserver process

In [ ]:
import subprocess, socket

### Getting runID
port  = 1234

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    sock.settimeout(5)
    result = sock.connect_ex(('localhost',port))

if result == 0:
    print('Server already running')
else:
    command = f"mlflow models serve -m runs:/{run.info.run_id}/{ARTIFACT_PATH} -p {port} --enable-mlserver"
    
    process = subprocess.Popen(command, shell=True)
    
    print(f'Executing: {command}')
    print(f"Started background process with PID: {process.pid} for model {run.info.run_id}/{ARTIFACT_PATH}")

Then we will query the endpoint using the next python cell

In [ ]:
import requests

url = f'http://localhost:{port}/invocations'

headers = {'Content-Type': 'application/json'}

data = {"inputs": [[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]]}

try:
    response = requests.post(url,headers=headers, json=data)
    print(f'Status Code: {response.status_code}, Result: {response.json()}')
except Exception as e:
    print(f'ERROR!!: An expected error happened: {e}')

## Deploying the Model to KServe

#### Building a Docker Image

```bash
mlflow models build-docker -m runs:/[the_latest_run_id]/wine-quality_model -n [dockerhub_user]/mlflow-wine-classifier --enable-mlserver
```

#### Pushing the Docker Image
```bash
docker push [dockerhub_user]/mlflow-wine-classifier
```

#### Deploying a Inference Service

We will deploy a Inference Service in our kubernetes cluster with KServer



```bash

### Creating a namespace mlflow-kserve-test
kubectl create namespace mlflow-kserve-test

### Creatinga Inference Service

cat <<EOF | kubectl apply -f -
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "mlflow-wine-classifier"
  namespace: "mlflow-kserve-test"
spec:
  predictor:
    model:
      modelFormat:
        name: mlflow
      protocolVersion: v2
      storageUri: "gs://mlflow-wine-classifier/wine-quality_model"
EOF

```

### Checking the Inference Service

```bash
watch kubectl -n mlflow-kserve-test get inferenceservice mlflow-wine-classifier
```

After some minutes the output expected will be something similar to the one below:
```bash
NAME                     URL                                                            READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                      AGE
mlflow-wine-classifier   http://mlflow-wine-classifier.mlflow-kserve-test.example.com   True           100                              mlflow-wine-classifier-predictor-00001   2m27s
```

### Testing the deployment

We will use a node IP to test out the deploy due to the model is exposed by an istio-ingressgatway loadBalancer

Firslty, we wil lset up manually the bash variable INGRESS_HOST and INGRESS_PORT

```bash
export INGRESS_HOST=$(kubectl get nodes -o json | jq '.items[1].status.addresses[] | select(.type=="InternalIP") | .address' | sed 's/"//g') # Assuming that there is at least one worker
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
echo "Host: $INGRESS_HOST, Port: $INGRESS_PORT"
```


Finally, using the file [test-input.json](./test-input.json), we will test out the inference endpoint by executing the bash snippet below:

```bash
SERVICE_HOSTNAME=$(kubectl get inferenceservice mlflow-wine-classifier -n mlflow-kserve-test -o jsonpath='{.status.url}' | cut -d "/" -f 3)
curl -v -H "Host: ${SERVICE_HOSTNAME}" -H "Content-Type: application/json" -d @./test-input.json "http://${INGRESS_HOST}:${INGRESS_PORT}/v1/models/mlflow-wine-classifier:predict"

```

## Links

### PyENV instalaltion
* Easy guid to install PyENV in ubuntu, https://medium.com/@aashari/easy-to-follow-guide-of-how-to-install-pyenv-on-ubuntu-a3730af8d7f0 